It a heart dataset that we know nothing about so lets just read it first from directory and lets see what its made off

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import warnings
warnings.filterwarnings("ignore")

import template


In [ ]:
get_dir = os.getcwd()
data_dir = os.path.join(get_dir,"Datasets","heart.csv")
data = pd.read_csv(data_dir)

#DATA CLEANING

In [ ]:
data.head()

In [ ]:
data.dtypes

In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:
data_val = template.data_val(data)
zero_count = data_val.zero_count()
print(zero_count)

In [ ]:
data["fbs"].unique()

The number of zero counts is not equal to the total number of samples. This indicates that there are no columns with all zero values
though column "fbs" has 85% zero counts but for now keep this feature

In [ ]:
data_val.histplot_columns1(7,2,figuresize=(15,15))

In [ ]:
column_type = data_val.get_column_type()

In [ ]:
data_val.scatterplot1(7,2,figuresize=(15,15),column_type=column_type)

In [ ]:
numerical_column = []
for i in data.columns:
    if column_type[i] == "numerical":
        numerical_column.append(i)


In [ ]:
sns.pairplot(data = data_scaled[numerical_column])

In [ ]:
print(pd.DataFrame(list(column_type.items()), columns=["columns","type"]))

In [ ]:
schema = data_val.schema()
print(schema)

In [ ]:
sns.set_theme()
plt.figure(figsize=(12,6))
sns.boxplot(data=data)
plt.tight_layout()
plt.show()

In [ ]:
data_cleaned = data_val.remove_outliers()

In [ ]:
sns.set_theme()
plt.figure(figsize=(12,6))
sns.boxplot(data=data_cleaned)
plt.tight_layout()
plt.show()

In [ ]:
data_scaled = data_val.scale_dataset(scaler1=MinMaxScaler)

In [ ]:
sns.set_theme()
plt.figure(figsize=(12,6))
sns.boxplot(data=data_scaled)
plt.tight_layout()
plt.show()

In [ ]:
sns.kdeplot(data=data_scaled)

In [ ]:
data_val.histplot_columns1(7,2,figuresize=(15,15))

In [ ]:
data_scaled.head()

In [ ]:
data_scaled.head()

In [ ]:
corrmap = data_scaled.corr()
sns.set_theme(style="white")
plt.figure(figsize=(11,9))
mask = np.triu(np.ones_like(corrmap, dtype=bool))
cmap = sns.diverging_palette(15,260, as_cmap=True)
sns.heatmap(corrmap,cmap=cmap,mask=mask,annot=True)

In [ ]:
x = data.drop(columns="target")
y = data["target"]

In [ ]:
x_train1,x_test1,y_train1,y_test1 = train_test_split(x,y,test_size=0.2,random_state=42)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV
from sklearn import linear_modelfrom sklearn.linear_model import LarsCV
from sklearn.linear_model import LarsCV,Lasso,LassoCV,LassoLars,LassoLarsCV,LassoLarsIC,ARDRegression,BayesianRidge,MultiTaskElasticNet,MultiTaskElasticNetCV,MultiTaskLasso,MultiTaskLassoCV,HuberRegressor,QuantileRegressor,RANSACRegressor,TheilSenRegressor,GammaRegressor,PoissonRegressor,TweedieRegressor,PassiveAggressiveRegressor


In [ ]:
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold

min_features_to_select = 1  # Minimum number of features to consider
clf = LogisticRegression()
cv = StratifiedKFold(5)

rfecv = RFECV(
    estimator=clf,
    step=1,
    cv=cv,
    scoring="accuracy",
    min_features_to_select=min_features_to_select,
    n_jobs=2,
)
rfecv.fit(x_train1, y_train1)


In [ ]:
import time

from sklearn.linear_model import LassoLarsIC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

start_time = time.time()
lasso_lars_ic = make_pipeline(StandardScaler(), LassoLarsIC(criterion="aic")).fit(X, y)
fit_time = time.time() - start_time

In [ ]:
results = pd.DataFrame(
    {
        "alphas": lasso_lars_ic[-1].alphas_,
        "AIC criterion": lasso_lars_ic[-1].criterion_,
    }
).set_index("alphas")
alpha_aic = lasso_lars_ic[-1].alpha_

In [ ]:
lasso_lars_ic.set_params(lassolarsic__criterion="bic").fit(X, y)
results["BIC criterion"] = lasso_lars_ic[-1].criterion_
alpha_bic = lasso_lars_ic[-1].alpha_

In [ ]:
def highlight_min(x):
    x_min = x.min()
    return ["font-weight: bold" if v == x_min else "" for v in x]


results.style.apply(highlight_min)

In [ ]:
ax = results.plot()
ax.vlines(
    alpha_aic,
    results["AIC criterion"].min(),
    results["AIC criterion"].max(),
    label="alpha: AIC estimate",
    linestyles="--",
    color="tab:blue",
)
ax.vlines(
    alpha_bic,
    results["BIC criterion"].min(),
    results["BIC criterion"].max(),
    label="alpha: BIC estimate",
    linestyle="--",
    color="tab:orange",
)
ax.set_xlabel(r"$\alpha$")
ax.set_ylabel("criterion")
ax.set_xscale("log")
ax.legend()
_ = ax.set_title(
    f"Information-criterion for model selection (training time {fit_time:.2f}s)"
)

In [ ]:
from sklearn.linear_model import LassoCV

start_time = time.time()
model = make_pipeline(StandardScaler(), LassoCV(cv=20)).fit(X, y)
fit_time = time.time() - start_time

In [ ]:
import matplotlib.pyplot as plt

ymin, ymax = 2300, 3800
lasso = model[-1]
plt.semilogx(lasso.alphas_, lasso.mse_path_, linestyle=":")
plt.plot(
    lasso.alphas_,
    lasso.mse_path_.mean(axis=-1),
    color="black",
    label="Average across the folds",
    linewidth=2,
)
plt.axvline(lasso.alpha_, linestyle="--", color="black", label="alpha: CV estimate")

plt.ylim(ymin, ymax)
plt.xlabel(r"$\alpha$")
plt.ylabel("Mean square error")
plt.legend()
_ = plt.title(
    f"Mean square error on each fold: coordinate descent (train time: {fit_time:.2f}s)"
)

In [ ]:
from sklearn.linear_model import LassoLarsCV
import time

start_time = time.time()
model = make_pipeline(StandardScaler(), LassoLarsCV(cv=20)).fit(X, y)
fit_time = time.time() - start_time

In [ ]:
lasso = model[-1]
plt.semilogx(lasso.cv_alphas_, lasso.mse_path_, ":")
plt.semilogx(
    lasso.cv_alphas_,
    lasso.mse_path_.mean(axis=-1),
    color="black",
    label="Average across the folds",
    linewidth=2,
)
plt.axvline(lasso.alpha_, linestyle="--", color="black", label="alpha CV")

plt.ylim(ymin, ymax)
plt.xlabel(r"$\alpha$")
plt.ylabel("Mean square error")
plt.legend()
_ = plt.title(f"Mean square error on each fold: Lars (train time: {fit_time:.2f}s)")